In [4]:
!pip install -U qiskit qiskit-aer-gpu-cu11

# qrng_cuda.py
from math import ceil
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def qrng_cuda(n_bits: int, n_qubits: int = 32):
    """
    Generate n_bits of random data using Qiskit Aer on CUDA.
    - n_qubits controls how many random bits you get per shot (>=1).
    Returns: dict with 'bits' (str), 'int' (int), 'bytes' (bytes).
    """
    # Try GPU; fall back to CPU if not available
    sim_check = AerSimulator() # Create an instance to check available devices
    gpu_ok = 'GPU' in sim_check.available_devices()
    sim = AerSimulator(method="statevector",
                       device="GPU" if gpu_ok else "CPU",
                       cuStateVec_enable=True)

    shots = ceil(n_bits / n_qubits)

    qc = QuantumCircuit(n_qubits, n_qubits)
    qc.h(range(n_qubits))                    # put all qubits in |+>
    qc.measure(range(n_qubits), range(n_qubits))  # measure to random bits

    tqc = transpile(qc, sim)
    # memory=True gives per-shot bitstrings (least-significant bit is qubit 0)
    result = sim.run(tqc, shots=shots, memory=True).result()
    bitstrings = result.get_memory(0)        # list like ['0101...', '1010...', ...]

    # Concatenate and trim to requested length
    bitstream = ''.join(bitstrings)[:n_bits]

    as_int = int(bitstream, 2)
    as_bytes = as_int.to_bytes((n_bits + 7) // 8, byteorder='big')

    return {
        "bits": bitstream,
        "int": as_int,
        "bytes": as_bytes,
    }

if __name__ == "__main__":
    # EXAMPLE: 256 random bits (32 bytes)
    out = qrng_cuda(256, n_qubits=16)
    print("First 64 bits:", out["bits"][:64])
    print("As int:", out["int"])
    print("As 32 bytes (hex):", out["bytes"].hex())

First 64 bits: 1111010100110110100111011111100010010111111011000001001011010101
As int: 110913147920640788555886158422710252757433011594552377404535534933684429158214
As 32 bytes (hex): f5369df897ec12d5f514fb36c66b693ea33480db499a91d50afac7dd0e467746
